In [ ]:
import glob

from potoo.plot import *
from potoo.util import *
import sklearn

from cache import *
from constants import *
from datasets import *
from features import *
from load import *
from sp14.model import *
from util import *
from viz import *

figsize('inline_short');

In [ ]:
load = Load()
all_recs = load.recs(
    # limit=30,  # XXX Faster dev
    # limit=3000,  # XXX Faster dev
    datasets=[
        # 'peterson-field-guide',
        # 'recordings',
        # TODO TODO Cache transformed data across the other datasets, for more lightweight classifier eval
        # 'mlsp-2013',  # Cached
        # 'nips4b',  # Cached
        'birdclef-2015',
    ],
)
display(
    df_summary(all_recs),
    all_recs[:5],
)

,dataset,species,species_com_name,duration_s,samples_mb,samples_n,basename,species_longhand,id,path,filesize_b
dtype,object,category,category,float64,float64,int64,object,category,object,object,int64
sizeof,2.5 MiB,291.9 KiB,291.9 KiB,518.8 KiB,518.8 KiB,518.8 KiB,2.6 MiB,291.9 KiB,3.5 MiB,3.6 MiB,518.8 KiB
len,33201,33201,33201,33201,33201,33201,33201,33201,33201,33201,33201
count,33201,33201,33201,33201,33201,33201,33201,33201,33201,33201,33201
nunique,1,1,1,32818,32818,32818,33201,1,33201,33201,33004
mean,NaN,NaN,NaN,3.409e+01,1.434e+00,7.516e+05,NaN,NaN,NaN,NaN,3.006e+06
std,NaN,NaN,NaN,3.911e+01,1.645e+00,8.625e+05,NaN,NaN,NaN,NaN,3.450e+06
min,birdclef-2015,_UNK,Unknown,0.258,0.011,5696,00000-2015-amazon,Unknown - _UNK,birdclef-2015/organized/wav/00000-2015-amazon,birdclef-2015/organized/wav/00000-2015-amazon.wav,22828
25%,birdclef-2015,_UNK,Unknown,13.800,0.581,304382,08342-2014-amazon,Unknown - _UNK,birdclef-2015/organized/wav/08342-2014-amazon,birdclef-2015/organized/wav/08342-2014-amazon.wav,1217572
50%,birdclef-2015,_UNK,Unknown,25.100,1.050,552705,16663-2015-amazon,Unknown - _UNK,birdclef-2015/organized/wav/16663-2015-amazon,birdclef-2015/organized/wav/16663-2015-amazon.wav,2210862


,dataset,species,species_com_name,duration_s,samples_mb,samples_n,basename,species_longhand,id,path,filesize_b
0,birdclef-2015,_UNK,Unknown,1.47,0.062,32397,00000-2015-amazon,Unknown - _UNK,birdclef-2015/organized/wav/00000-2015-amazon,birdclef-2015/organized/wav/00000-2015-amazon.wav,129632
22143,birdclef-2015,_UNK,Unknown,54.00,2.270,1190908,22207-2015-amazon,Unknown - _UNK,birdclef-2015/organized/wav/22207-2015-amazon,birdclef-2015/organized/wav/22207-2015-amazon.wav,4763674
22142,birdclef-2015,_UNK,Unknown,169.00,7.090,3717497,22206-2015-amazon,Unknown - _UNK,birdclef-2015/organized/wav/22206-2015-amazon,birdclef-2015/organized/wav/22206-2015-amazon.wav,14870030
22141,birdclef-2015,_UNK,Unknown,75.30,3.170,1661045,22205-2015-amazon,Unknown - _UNK,birdclef-2015/organized/wav/22205-2015-amazon,birdclef-2015/organized/wav/22205-2015-amazon.wav,6644224
22140,birdclef-2015,_UNK,Unknown,9.95,0.418,219319,22204-2015-amazon,Unknown - _UNK,birdclef-2015/organized/wav/22204-2015-amazon,birdclef-2015/organized/wav/22204-2015-amazon.wav,877320


In [ ]:
# Stats
(all_recs
    .assign(
        n=1,
        duration_h=lambda df: df.duration_s / 3600,
        samples_gb=lambda df: df.samples_mb / 1024,
        species=lambda df: df.species.cat.remove_unused_categories(),  # Else groupby includes all categories
    )
    .groupby([
        'dataset',
        'species',
    ])
    [['n', 'duration_h', 'samples_gb', 'samples_n']]
    .sum()
    .dropna()
)

,,n,duration_h,samples_gb,samples_n
dataset,species,,,,
birdclef-2015,_UNK,33201,314.0,46.5,24954119550


In [ ]:
# (all_recs
#     .assign(species=lambda df: df.species_longhand)
#     .assign(n=1)
#     .pipe(lambda df: df.append(
#         pd.DataFrame([
#             dict(species=species, dataset=dataset, n=0)
#             for species in df.species.unique()
#             for dataset in df.dataset.unique()
#         ])
#         .astype({'species': df.species.dtype})
#     ))
#     .groupby(['dataset', 'species'])['n'].sum().reset_index()
#     .pipe(df_reverse_cat, 'species')
#     .pipe(ggplot, aes(x='species', y='n', color='dataset', fill='dataset'))
#     + coord_flip()
#     + stat_identity(geom='bar', position=position_dodge())
#     + ylab('num recordings')
#     + scale_color_cmap_d(mpl.cm.tab10)
#     + scale_fill_cmap_d(mpl.cm.tab10)
#     + theme_figsize('half')
#     + ggtitle('How many all_recs per species x dataset?')
# )

# Fit search

In [ ]:
# Load models
projection = Projection.load('peterson-v0-26bae1c')
features = projection.features
load = features.load

[19:00:45.393] Projection.load
  path: /Users/danb/hack/bubo/data/models/projection/peterson-v0-26bae1c.pkl


In [ ]:
# TODO TODO Cache transformed data across the other datasets, for more lightweight classifier iteration

In [ ]:
%%prun -l5
# TODO TODO Re-run overnight on full data (only ~50% was cached on Tue night, because it failed partway through)
#   - TODO Maybe need to refresh a junky cache? -- conclude 'no' if nothing barfs
# Add features
# all_recs = features.transform(all_recs)  # Not mem safe [TODO TODO Re-enable after birdclef]
with cache_control():
    recs = projection.transform(all_recs
        .sample(random_state=0,
            frac=.001,  # ~30, cached[.1s], refresh[3.7s]
            # frac=.01,  # ~300, cached[.6s], refresh[39s]
            # frac=.1,  # ~3K, cached[5.5s], refresh[TODO ~390s]
            # frac=1,  # ~33K, cached[TODO], refresh[TODO ~3900s]
        )
    )

[                                                                                          ] | 0% Completed |  0.0s

[                                                                                          ] | 0% Completed |  0.1s

[                                                                                          ] | 0% Completed |  0.2s

[                                                                                          ] | 0% Completed |  0.3s

[                                                                                          ] | 0% Completed |  0.4s

...

[########################################################################                  ] | 80% Completed |  5.2s

[########################################################################                  ] | 80% Completed |  5.3s

[##########################################################################                ] | 82% Completed |  5.4s

[##########################################################################################] | 100% Completed |  5.5s

         613435 function calls (597444 primitive calls) in 6.064 seconds

   Ordered by: internal time
   List reduced from 999 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      951    5.435    0.006    5.435    0.006 {method 'acquire' of '_thread.lock' objects}
        4    0.099    0.025    0.247    0.062 base.py:609(tokenize)
      168    0.048    0.000    0.048    0.000 socket.py:333(send)
        4    0.047    0.012    0.047    0.012 {built-in method _hashlib.openssl_md5}
   132995    0.037    0.000    0.074    0.000 {built-in method builtins.isinstance}

In [ ]:
df_summary(recs).T

,,,,,,,,min,25%,50%,75%,max
,dtype,sizeof,len,count,nunique,mean,std,,,,,
dataset,object,594.5 KiB,3320,3320,1.0,NaN,NaN,birdclef-2015,birdclef-2015,birdclef-2015,birdclef-2015,birdclef-2015
species,category,189.3 KiB,3320,3320,1.0,NaN,NaN,_UNK,_UNK,_UNK,_UNK,_UNK
species_com_name,category,189.3 KiB,3320,3320,1.0,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown
duration_s,float64,211.9 KiB,3320,3320,3316.0,3.476e+01,3.663e+01,0.506,13.7,25.2,44.3,744
samples_mb,float64,211.9 KiB,3320,3320,3316.0,1.462e+00,1.540e+00,0.0213,0.576,1.06,1.86,31.3
samples_n,int64,211.9 KiB,3320,3320,3316.0,7.664e+05,8.076e+05,11163,302148,556386,975787,16404677
basename,object,426.2 KiB,3320,3320,3320.0,NaN,NaN,00012-2014-amazon,08341-2014-amazon,16474-2015-amazon,24692-2015-amazon,33277-2015-amazon
species_longhand,category,189.4 KiB,3320,3320,1.0,NaN,NaN,Unknown - _UNK,Unknown - _UNK,Unknown - _UNK,Unknown - _UNK,Unknown - _UNK
id,object,517.6 KiB,3320,3320,3320.0,NaN,NaN,birdclef-2015/organized/wav/00012-2014-amazon,birdclef-2015/organized/wav/08341-2014-amazon,birdclef-2015/organized/wav/16474-2015-amazon,birdclef-2015/organized/wav/24692-2015-amazon,birdclef-2015/organized/wav/33277-2015-amazon


In [ ]:
# We've got 3K feats!
display(
    recs[:10].feat.map(lambda x: x.shape),
    recs[:10].feat,
)

23949    (1500,)
22621    (1500,)
7795     (1500,)
29069    (1500,)
31947    (1500,)
1882     (1500,)
4128     (1500,)
8438     (1500,)
25012    (1500,)
10168    (1500,)
Name: feat, dtype: object

23949    [0.06331406909893861, -0.04964070270613392, -0.19131248028733425, -0.05824633627854145, -0.0234890567855365, -0.047526276340414085, 0.05807471691779061, -0.05800694005496489, 0.16503591515584703, ...
22621    [0.042141750406272965, -0.08074065630429209, -0.03281943936089645, -0.023370947752462127, -0.024991005228055065, -0.11012978069028238, -0.026134645222657637, -0.014313489008350664, 0.0936649871631...
7795     [0.03542259352817756, -0.06467540066504998, -0.1338321972752299, -0.048604736173224763, -0.052808958661699046, -0.1216442274584481, 0.03861972528229087, -0.07368314775001054, 0.057084450107732915,...
29069    [-0.0527608431988142, -0.058531699163404685, -0.09129230601447097, -0.26304096043639535, -0.11491441396519753, -0.12470168093905273, -0.09494553642944675, -0.002355762970141453, 0.2753330935660243...
31947    [0.016979301800376795, -0.07029390929782203, -0.024902013221959764, -0.08544627385359833, -0.0776335656720306, -0.1011209723311656, 0.03197187869602822, -0.05587345530940274, 0.10091512238575258, ...
1882     [0.05550616643231118, -0.03519901266370513, -0.019293795270201523, -0.0354109395174347, -0.05321374931130233, 0.0023116204608399533, 0.06095776508442117, -0.047717688071486485, 0.00648224720333399...
4128     [0.13534282883285711, -0.05536386093894452, -0.26039399284763026, -0.008527874984285155, -0.02191793989762187, -0.04522016202520316, 0.1040224402863438, 0.05886787188823713, 0.09950979271566598, -...
8438     [0.03894124691596049, -0.12286858095619413, -0.10941534859890337, -0.07178473713253128, -0.14642008559388942, -0.17162744939188587, 0.007582474726178153, 0.36765842426072626, 0.027473735512101586,...
25012    [0.015989996294293363, -0.07095293690526452, -0.17932717543355853, 0.03792351736090592, -0.01177335669671723, -0.0332569015095621, 0.08965888096206956, -0.08339860667403058, 0.07224692881628633, -...
10168    [0.062358262261663425, -0.09501595563603349, -0.11856837350426165, -0.013375328525719331, -0.0311702746954676, -0.06245895055741846, 0.06778157581520124, -0.07539310515357862, 0.051207645146234305...
Name: feat, dtype: object

In [ ]:
# TODO TODO Read metadata for birdclef
# TODO TODO Eval classifiers

In [ ]:
recs_eval = (recs
    [lambda df: df.dataset == 'peterson-field-guide']
    # [:10]  # Faster dev
    .reset_index(drop=True)
)
train_n, test_n = (len(recs_eval)+1)//2, len(recs_eval)//2
recs_train, recs_test = (recs_eval
    .pipe(sklearn.utils.shuffle, random_state=0)
    .sample(train_n + test_n, random_state=0)
    .pipe(lambda df: (
        df[:train_n],
        df[train_n : train_n + test_n],
    ))
)
log('params', **{
    'recs_eval': len(recs_eval),
    'recs_train': len(recs_train),
    'recs_test': len(recs_test),
})

In [ ]:
%%time
from sp14.model import *
search = Search(
    projection=projection,
    # n_neighbors=3,
    n_neighbors=5,
    # n_neighbors=10,
)
search.fit(recs_train)

# Eval search

In [ ]:
(search.coverage_error(recs_test, by='species')
    .pipe(lambda df: df_transform_cat(df, lambda _: df.sort_values('coverage_error').species, 'species'))
    .pipe(ggplot, aes(x='species', y='coverage_error'))
    + geom_point()
    + coord_flip()
    + geom_hline(yintercept=len(set(search.fit_classes_)), color='grey')
    + theme_figsize('inline')
    + ggtitle('Coverage error')
)

In [ ]:
%%time
with with_figsize('full'):
    search.plot_confusion_matrix(recs_test)

In [ ]:
# Example usage
# search.species(recs_test[:5])
# search.species_probs(recs_test[:5]).T[:search.knn_.n_neighbors + 1].T
# search.similar_recs(recs_test[:5], 10)